In [30]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [93]:
source('main.R')

data <- kaggle.house$loadData()

df.training <- data$train
df.testing <- data$test
df.combined <- kaggle.house$getCombinedDataset(df.training, df.testing)

df.combined.bad.na.fixed <- kaggle.house$na$fixAll(df.combined)

df.training.bad.na.fixed <- 
    df.combined.bad.na.fixed %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource)
    
df.testing.bad.na.fixed  <- 
    df.combined.bad.na.fixed %>% 
    filter(dataSource == "test") %>% 
    select(-dataSource)

res <- kaggle.house$trans$type2TransContainer$BldgType(df.training.bad.na.fixed)

res$df.new %>% select(BldgType, building_type) %>% head
testsetTransformator <- res$testsetTransformator
testsetTransformator(df.testing.bad.na.fixed) %>% select(BldgType, building_type) %>% head

BldgType,building_type
1Fam,12.03112
1Fam,12.03112
1Fam,12.03112
1Fam,12.03112
1Fam,12.03112
1Fam,12.03112


BldgType,building_type
1Fam,10.46024
1Fam,10.46024
1Fam,10.46024
1Fam,10.46024
TwnhsE,11.23189
1Fam,10.46024


In [49]:
allCategVars <- kaggle.house$getCategoricalColumnNames(df.combined.bad.na.fixed %>% select(-dataSource)) %>% sort
type1CategVars <- kaggle.house$trans$type1TransContainer %>% names %>% sort
type2CategVars <- kaggle.house$trans$type2TransContainer %>% names %>% sort

stopifnot(intersect(type1CategVars, type2CategVars) %>% length == 0)

setdiff(allCategVars, union(type1CategVars, type2CategVars))

character(0)

In [33]:
library(IRdisplay)

type1 <- function(attrName, newAttrName) {
        
    attrName <- enquo(attrName)
    newAttrName <- enquo(newAttrName)
    
    tranFun <- kaggle.house$trans$type1TransContainer[[as.character(attrName)[2]]]
    
    trainBefore <- df.training.bad.na.fixed %>% 
                   group_by(!!attrName) %>% count %>% prepare_mimebundle 
    trainAfter  <- tranFun(df.training.bad.na.fixed) %>% 
                   group_by(!!newAttrName, !!attrName) %>% count %>% prepare_mimebundle
    
    testBefore <- df.testing.bad.na.fixed %>% 
                  group_by(!!attrName) %>% count %>% prepare_mimebundle
    testAfter <- tranFun(df.testing.bad.na.fixed) %>% 
                 group_by(!!newAttrName, !!attrName) %>% count %>% prepare_mimebundle

    z <- paste0('<table>
                  <tr style="background: transparent">', 
                   '<td style="padding-right: 20px"><b>train before</b>', trainBefore$data[['text/html']], '</td>',
                   '<td style="padding-right: 20px"><b>train after</b>',  trainAfter$data[['text/html']],  '</td>',
                   '<td style="padding-right: 20px"><b>test before</b>',  testBefore$data[['text/html']],  '</td>',
                   '<td><b>test before</b>',                              testAfter$data[['text/html']],   '</td>',
                 '</tr>
                </table>')
    
    trainBefore$data[['text/html']] <- z
    publish_mimebundle(trainBefore$data, trainBefore$metadata)
}

In [34]:
type1(Alley, has_alley_access)
type1(CentralAir, has_central_air)

Alley n   
1 Grvl    50
2 Pave    41
3 NA    1369

CentralAir n   
1 N            95
2 Y          1365

In [44]:
type1(Electrical, standard_electrical)
type1(Functional, is_full_functional)

Electrical n   
1 FuseA        94
2 FuseF        27
3 FuseP         3
4 Mix           1
5 SBrkr      1335

Functional n   
1 Maj1         14
2 Maj2          5
3 Min1         31
4 Min2         34
5 Mod          15
6 Sev           1
7 Typ        1360

In [45]:
type1(Heating, heating_air_furnace)
type1(LandContour, is_land_level)

Heating n   
1 Floor      1
2 GasA    1428
3 GasW      18
4 Grav       7
5 OthW       2
6 Wall       4

LandContour n   
1 Bnk           63
2 HLS           50
3 Low           36
4 Lvl         1311

In [46]:
type1(LandSlope, is_slope)
type1(LotShape, is_lotshape_regular)

LandSlope n   
1 Gtl       1382
2 Mod         65
3 Sev         13

LotShape n  
1 IR1      484
2 IR2       41
3 IR3       10
4 Reg      925

In [47]:
type1(MiscFeature, has_misc_feature)
type1(PavedDrive, has_paved_drive)

MiscFeature n   
1 Gar2           2
2 Othr           2
3 Shed          49
4 TenC           1
5 NA          1406

PavedDrive n   
1 N            90
2 P            30
3 Y          1340

In [121]:
type1(PoolQC, has_pool)
type1(RoofMatl, standard_roof_material)

PoolQC n   
1 Ex        2
2 Fa        2
3 Gd        3
4 NA     1453

RoofMatl n   
1 ClyTile     1
2 CompShg  1434
3 Membran     1
4 Metal       1
5 Roll        1
6 Tar&Grv    11
7 WdShake     5
8 WdShngl     6

In [122]:
type1(Street, is_street_paved)
type1(Utilities, all_utilities)

Street n   
1 Grvl      6
2 Pave   1454

Utilities n   
1 AllPub    1459
2 NoSeWa       1

In [89]:
library(IRdisplay)

type2 <- function(attrName, newAttrName) {
        
    attrName <- enquo(attrName)
    newAttrName <- enquo(newAttrName)
    
    tranFun <- kaggle.house$trans$type2TransContainer[[as.character(attrName)[2]]]
    
    result = tranFun(df.training.bad.na.fixed)
    
    df.trainAfter <- result$df.new
    testsetTransformator <- result$testsetTransformator
    df.testAfter <- testsetTransformator(df.testing.bad.na.fixed)
    
    x.train <- 
        df.trainAfter %>% 
        group_by(!!attrName) %>%
        summarise(mi = min(!!newAttrName), ma = max(!!newAttrName), n = n()) %>% 
        mutate(mi == ma) %>% prepare_mimebundle
    
    x.test <- 
        df.trainAfter %>% 
        group_by(!!attrName) %>%
        summarise(mi = min(!!newAttrName), ma = max(!!newAttrName), n = n()) %>% 
        mutate(mi == ma) %>% prepare_mimebundle

    z <- paste0('<table>
                  <tr style="background: transparent">', 
                   '<td style="padding-right: 20px"><b>train</b>', x.train$data[['text/html']], '</td>',
                   '<td><b>test</b>',                              x.test$data[['text/html']],   '</td>',
                 '</tr>
                </table>')
    
    x.train$data[['text/html']] <- z
    publish_mimebundle(x.train$data, x.train$metadata)
}

In [91]:
type2(BldgType, building_type)
#type2(BsmtCond, basement_condition)

BldgType,mi,ma,n,mi == ma
1Fam,12.03112,12.03112,1220,TRUE
2fmCon,11.75587,11.75587,31,TRUE
Duplex,11.82026,11.82026,52,TRUE
Twnhs,11.83138,11.83138,43,TRUE
TwnhsE,12.05641,12.05641,114,TRUE
BldgType,mi,ma,n,mi == ma
1Fam,12.03112,12.03112,1220,TRUE
2fmCon,11.75587,11.75587,31,TRUE
Duplex,11.82026,11.82026,52,TRUE
Twnhs,11.83138,11.83138,43,TRUE


In [60]:
type2(BsmtExposure, basement_exposure)
type2(BsmtFinType1, basement_finish1)

BsmtExposure,mi,ma,n,mi == ma
Av,12.13270,12.13270,221,TRUE
Gd,12.33256,12.33256,134,TRUE
Mn,12.11413,12.11413,114,TRUE
No,11.94471,11.94471,953,TRUE
NA,11.55216,11.55216,38,TRUE
BsmtExposure,mi,ma,n,mi == ma
Av,12.13270,12.13270,221,TRUE
Gd,12.33256,12.33256,134,TRUE
Mn,12.11413,12.11413,114,TRUE
No,11.94471,11.94471,953,TRUE


BsmtFinType1,mi,ma,n,mi == ma
ALQ,11.91338,11.91338,220,TRUE
BLQ,11.84295,11.84295,148,TRUE
GLQ,12.27256,12.27256,418,TRUE
LwQ,11.84223,11.84223,74,TRUE
Rec,11.86358,11.86358,133,TRUE
Unf,11.99381,11.99381,430,TRUE
NA,11.53077,11.53077,37,TRUE
BsmtFinType1,mi,ma,n,mi == ma
ALQ,11.91338,11.91338,220,TRUE
BLQ,11.84295,11.84295,148,TRUE


In [62]:
type2(BsmtFinType2, basement_finish2)
type2(BsmtQual, basement_height_quality)

BsmtFinType2,mi,ma,n,mi == ma
ALQ,12.07197,12.07197,19,TRUE
BLQ,11.87060,11.87060,33,TRUE
GLQ,12.22033,12.22033,14,TRUE
LwQ,11.94469,11.94469,46,TRUE
Rec,11.90995,11.90995,54,TRUE
Unf,12.02575,12.02575,1256,TRUE
NA,11.55216,11.55216,38,TRUE
BsmtFinType2,mi,ma,n,mi == ma
ALQ,12.07197,12.07197,19,TRUE
BLQ,11.87060,11.87060,33,TRUE


BsmtQual,mi,ma,n,mi == ma
Ex,12.66981,12.66981,121,TRUE
Fa,11.62625,11.62625,35,TRUE
Gd,12.16562,12.16562,618,TRUE
TA,11.81673,11.81673,649,TRUE
NA,11.53077,11.53077,37,TRUE
BsmtQual,mi,ma,n,mi == ma
Ex,12.66981,12.66981,121,TRUE
Fa,11.62625,11.62625,35,TRUE
Gd,12.16562,12.16562,618,TRUE
TA,11.81673,11.81673,649,TRUE


In [12]:
type2(Condition1, cond1)
type2(Condition2, cond2)

Condition1,mi,ma,n,mi == ma
Artery,11.69149,11.69149,48,TRUE
Feedr,11.84940,11.84940,81,TRUE
Norm,12.02275,12.02275,1260,TRUE
PosA,12.26106,12.26106,8,TRUE
PosN,12.20607,12.20607,19,TRUE
RRAe,11.86710,11.86710,11,TRUE
RRAn,12.05227,12.05227,26,TRUE
RRNe,12.15853,12.15853,2,TRUE
RRNn,12.27373,12.27373,5,TRUE
Condition1,mi,ma,n,mi == ma


Condition2,mi,ma,n,mi == ma
Artery,11.57004,11.57004,2,TRUE
Feedr,11.75586,11.75586,6,TRUE
Norm,12.00457,12.00457,1445,TRUE
PosA,12.69158,12.69158,1,TRUE
PosN,12.49388,12.49388,2,TRUE
RRAe,12.15478,12.15478,1,TRUE
RRAn,11.82704,11.82704,1,TRUE
RRNn,11.43533,11.43533,2,TRUE
Condition2,mi,ma,n,mi == ma
Artery,11.57004,11.57004,2,TRUE


In [28]:
type2(ExterCond, exter_cond)
type2(ExterQual, exter_qual)

ExterCond,mi,ma,n,mi == ma
Ex,11.98916,11.98916,3,TRUE
Fa,11.46736,11.46736,28,TRUE
Gd,11.92669,11.92669,146,TRUE
Po,11.24505,11.24505,1,TRUE
TA,12.02796,12.02796,1282,TRUE
ExterCond,mi,ma,n,mi == ma
Ex,11.98916,11.98916,3,TRUE
Fa,11.46736,11.46736,28,TRUE
Gd,11.92669,11.92669,146,TRUE
Po,11.24505,11.24505,1,TRUE


ExterQual,mi,ma,n,mi == ma
Ex,12.80655,12.80655,52,TRUE
Fa,11.31751,11.31751,14,TRUE
Gd,12.30138,12.30138,488,TRUE
TA,11.84546,11.84546,906,TRUE
ExterQual,mi,ma,n,mi == ma
Ex,12.80655,12.80655,52,TRUE
Fa,11.31751,11.31751,14,TRUE
Gd,12.30138,12.30138,488,TRUE
TA,11.84546,11.84546,906,TRUE


In [29]:
type2(Exterior1st, exter_1)
type2(Exterior2nd, exter_2)

Exterior1st,mi,ma,n,mi == ma
AsbShng,11.58972,11.58972,20,TRUE
AsphShn,11.51293,11.51293,1,TRUE
BrkComm,11.15829,11.15829,2,TRUE
BrkFace,12.01823,12.01823,50,TRUE
CBlock,11.56172,11.56172,1,TRUE
CemntBd,12.37370,12.37370,61,TRUE
HdBoard,11.91772,11.91772,222,TRUE
ImStucc,12.47610,12.47610,1,TRUE
MetalSd,11.84223,11.84223,220,TRUE
Plywood,12.02844,12.02844,108,TRUE


Exterior2nd,mi,ma,n,mi == ma
AsbShng,11.61724,11.61724,20,TRUE
AsphShn,11.84223,11.84223,3,TRUE
Brk Cmn,11.89819,11.89819,7,TRUE
BrkFace,11.98293,11.98293,25,TRUE
CBlock,11.56172,11.56172,1,TRUE
CmentBd,12.38313,12.38313,60,TRUE
HdBoard,11.95118,11.95118,207,TRUE
ImStucc,12.14106,12.14106,10,TRUE
MetalSd,11.84043,11.84043,214,TRUE
Other,12.67295,12.67295,1,TRUE


In [138]:
type2(Fence, fence)
type2(FireplaceQu, fireplace_qual)
type2(Foundation, foundation)

Fence,mi,ma,n,mi == ma
GdPrv,12.02874,12.02874,59,TRUE
GdWo,11.84043,11.84043,54,TRUE
MnPrv,11.83102,11.83102,157,TRUE
MnWw,11.77529,11.77529,11,TRUE
NA,12.06105,12.06105,1179,TRUE
Fence,mi,ma,n,mi == ma
GdPrv,12.02874,12.02874,59,TRUE
GdWo,11.84043,11.84043,54,TRUE
MnPrv,11.83102,11.83102,157,TRUE
MnWw,11.77529,11.77529,11,TRUE


FireplaceQu,mi,ma,n,mi == ma
Ex,12.65794,12.65794,24,TRUE
Fa,11.97035,11.97035,33,TRUE
Gd,12.24023,12.24023,380,TRUE
Po,11.78658,11.78658,20,TRUE
TA,12.14153,12.14153,313,TRUE
NA,11.81303,11.81303,690,TRUE
FireplaceQu,mi,ma,n,mi == ma
Ex,12.65794,12.65794,24,TRUE
Fa,11.97035,11.97035,33,TRUE
Gd,12.24023,12.24023,380,TRUE


Foundation,mi,ma,n,mi == ma
BrkTil,11.73807,11.73807,146,TRUE
CBlock,11.86005,11.86005,634,TRUE
PConc,12.23077,12.23077,647,TRUE
Slab,11.55333,11.55333,24,TRUE
Stone,11.74454,11.74454,6,TRUE
Wood,12.00762,12.00762,3,TRUE
Foundation,mi,ma,n,mi == ma
BrkTil,11.73807,11.73807,146,TRUE
CBlock,11.86005,11.86005,634,TRUE
PConc,12.23077,12.23077,647,TRUE


In [82]:
type2(GarageCond, garage_cond)
type2(GarageFinish, garage_finish)
type2(GarageQual, garage_qual)
type2(GarageType, garage_type)

GarageFinish,mi,ma,n,mi == ma
Fin,12.27839,12.27839,352,TRUE
RFn,12.15478,12.15478,422,TRUE
Unf,11.81303,11.81303,605,TRUE
NA,11.51293,11.51293,81,TRUE
GarageFinish,mi,ma,n,mi == ma
Fin,12.27839,12.27839,352,TRUE
RFn,12.15478,12.15478,422,TRUE
Unf,11.81303,11.81303,605,TRUE
NA,11.51293,11.51293,81,TRUE


GarageType,mi,ma,n,mi == ma
2Types,11.97505,11.97505,6,TRUE
Attchd,12.12811,12.12811,870,TRUE
Basment,11.90497,11.90497,19,TRUE
BuiltIn,12.33485,12.33485,88,TRUE
CarPort,11.58989,11.58989,9,TRUE
Detchd,11.77144,11.77144,387,TRUE
NA,11.51293,11.51293,81,TRUE
GarageType,mi,ma,n,mi == ma
2Types,11.97505,11.97505,6,TRUE
Attchd,12.12811,12.12811,870,TRUE


In [88]:
type2(HeatingQC, heating_qual)
type2(HouseStyle, house_style)

HeatingQC,mi,ma,n,mi == ma
Ex,12.17922,12.17922,741,TRUE
Fa,11.72400,11.72400,49,TRUE
Gd,11.93164,11.93164,241,TRUE
Po,11.37366,11.37366,1,TRUE
TA,11.81303,11.81303,428,TRUE
HeatingQC,mi,ma,n,mi == ma
Ex,12.17922,12.17922,741,TRUE
Fa,11.72400,11.72400,49,TRUE
Gd,11.93164,11.93164,241,TRUE
Po,11.37366,11.37366,1,TRUE


HouseStyle,mi,ma,n,mi == ma
1.5Fin,11.79055,11.79055,154,TRUE
1.5Unf,11.61941,11.61941,14,TRUE
1Story,11.94956,11.94956,726,TRUE
2.5Fin,12.17549,12.17549,8,TRUE
2.5Unf,11.80485,11.80485,11,TRUE
2Story,12.15478,12.15478,445,TRUE
SFoyer,11.82012,11.82012,37,TRUE
SLvl,12.01067,12.01067,65,TRUE
HouseStyle,mi,ma,n,mi == ma
1.5Fin,11.79055,11.79055,154,TRUE


In [98]:
type2(KitchenQual, kitchen_qual)
type2(LotConfig, lot_config)

KitchenQual,mi,ma,n,mi == ma
Ex,12.66586,12.66586,100,TRUE
Fa,11.65269,11.65269,39,TRUE
Gd,12.21305,12.21305,586,TRUE
TA,11.82774,11.82774,735,TRUE
KitchenQual,mi,ma,n,mi == ma
Ex,12.66586,12.66586,100,TRUE
Fa,11.65269,11.65269,39,TRUE
Gd,12.21305,12.21305,586,TRUE
TA,11.82774,11.82774,735,TRUE


LotConfig,mi,ma,n,mi == ma
Corner,11.98293,11.98293,263,TRUE
CulDSac,12.20235,12.20235,94,TRUE
FR2,12.01370,12.01370,47,TRUE
FR3,12.18131,12.18131,4,TRUE
Inside,11.98104,11.98104,1052,TRUE
LotConfig,mi,ma,n,mi == ma
Corner,11.98293,11.98293,263,TRUE
CulDSac,12.20235,12.20235,94,TRUE
FR2,12.01370,12.01370,47,TRUE
FR3,12.18131,12.18131,4,TRUE


In [106]:
type2(MasVnrType, masonry_veneer_type)
type2(MSSubClass, ms_sub_class)

MasVnrType,mi,ma,n,mi == ma
BrkCmn,11.84223,11.84223,15,TRUE
BrkFace,12.10625,12.10625,445,TRUE
None,11.87060,11.87060,864,TRUE
Stone,12.41647,12.41647,128,TRUE
NA,12.22229,12.22229,8,TRUE
MasVnrType,mi,ma,n,mi == ma
BrkCmn,11.84223,11.84223,15,TRUE
BrkFace,12.10625,12.10625,445,TRUE
None,11.87060,11.87060,864,TRUE
Stone,12.41647,12.41647,128,TRUE


MSSubClass,mi,ma,n,mi == ma
120,12.16525,12.16525,87,TRUE
160,11.89136,11.89136,63,TRUE
180,11.39036,11.39036,10,TRUE
190,11.76172,11.76172,30,TRUE
20,11.97823,11.97823,536,TRUE
30,11.51192,11.51192,69,TRUE
40,11.86487,11.86487,4,TRUE
45,11.58507,11.58507,12,TRUE
50,11.79055,11.79055,144,TRUE
60,12.27932,12.27932,299,TRUE


In [109]:
type2(MSZoning, ms_zoning)
type2(Neighborhood, nbhood)

MSZoning,mi,ma,n,mi == ma
C (all),11.21767,11.21767,10,TRUE
FV,12.23539,12.23539,65,TRUE
RH,11.82375,11.82375,16,TRUE
RL,12.06681,12.06681,1151,TRUE
RM,11.69941,11.69941,218,TRUE
MSZoning,mi,ma,n,mi == ma
C (all),11.21767,11.21767,10,TRUE
FV,12.23539,12.23539,65,TRUE
RH,11.82375,11.82375,16,TRUE
RL,12.06681,12.06681,1151,TRUE


Neighborhood,mi,ma,n,mi == ma
Blmngtn,12.16003,12.16003,17,TRUE
Blueste,11.82654,11.82654,2,TRUE
BrDale,11.57119,11.57119,16,TRUE
BrkSide,11.73022,11.73022,58,TRUE
ClearCr,12.20732,12.20732,28,TRUE
CollgCr,12.19197,12.19197,150,TRUE
Crawfor,12.20919,12.20919,51,TRUE
Edwards,11.70962,11.70962,100,TRUE
Gilbert,12.10625,12.10625,79,TRUE
IDOTRR,11.54248,11.54248,37,TRUE


In [112]:
type2(RoofStyle, roof_style)
type2(SaleCondition, sale_cond)

RoofStyle,mi,ma,n,mi == ma
Flat,12.12811,12.12811,13,TRUE
Gable,11.98293,11.98293,1141,TRUE
Gambrel,11.84223,11.84223,11,TRUE
Hip,12.08108,12.08108,286,TRUE
Mansard,12.07254,12.07254,7,TRUE
Shed,12.31161,12.31161,2,TRUE
RoofStyle,mi,ma,n,mi == ma
Flat,12.12811,12.12811,13,TRUE
Gable,11.98293,11.98293,1141,TRUE
Gambrel,11.84223,11.84223,11,TRUE


SaleCondition,mi,ma,n,mi == ma
Abnorml,11.77529,11.77529,101,TRUE
AdjLand,11.52926,11.52926,4,TRUE
Alloca,11.90533,11.90533,12,TRUE
Family,11.85245,11.85245,20,TRUE
Normal,11.98293,11.98293,1198,TRUE
Partial,12.40738,12.40738,125,TRUE
SaleCondition,mi,ma,n,mi == ma
Abnorml,11.77529,11.77529,101,TRUE
AdjLand,11.52926,11.52926,4,TRUE
Alloca,11.90533,11.90533,12,TRUE


In [117]:
type2(SaleType, sale_type)

SaleType,mi,ma,n,mi == ma
COD,11.84223,11.84223,43,TRUE
Con,12.48391,12.48391,2,TRUE
ConLD,11.84940,11.84940,9,TRUE
ConLI,11.73607,11.73607,5,TRUE
ConLw,11.87757,11.87757,5,TRUE
CWD,12.13473,12.13473,4,TRUE
New,12.41897,12.41897,122,TRUE
Oth,11.66178,11.66178,3,TRUE
WD,11.97035,11.97035,1267,TRUE
SaleType,mi,ma,n,mi == ma
